In [ ]:
from paraview.simple import *
import numpy as np

In [ ]:
RequestInfo = """
import numpy as np
executive = self.GetExecutive ()
outInfo = executive.GetOutputInformation(0)

dims = [24,24,24]
outInfo.Set(executive.WHOLE_EXTENT(), 0, dims[0]-1 , 0, dims[1]-1 , 0, dims[2]-1)
outInfo.Set(vtk.vtkDataObject.SPACING(), 1, 1, 1)
outInfo.Set(vtk.vtkDataObject.ORIGIN(), 0,0,0)

outInfo.Set(vtk.vtkAlgorithm.CAN_PRODUCE_SUB_EXTENT(), 1)

# advertize that we can produce N timesteps, and give the timestep values
timesteps = np.arange(32)*.1
outInfo.Remove(executive.TIME_STEPS())
for timestep in timesteps:
  outInfo.Append(executive.TIME_STEPS(), timestep)
outInfo.Remove(executive.TIME_RANGE())
outInfo.Append(executive.TIME_RANGE(), timesteps[0])
outInfo.Append(executive.TIME_RANGE(), timesteps[-1])
"""

In [ ]:
RequestData = """
import numpy as np
executive = self.GetExecutive()
outInfo = executive.GetOutputInformation(0)
# find out the extents requested by the downstream pipeline
exts = [executive.UPDATE_EXTENT().Get(outInfo, i) for i in range(6)]
# find out the timestep value requested by the downstream pipeline
ts = executive.UPDATE_TIME_STEP().Get(outInfo)

dims = (exts[1]-exts[0]+1, exts[3]-exts[2]+1, exts[5]-exts[4]+1)
# create a grid of given dimensions 
output.SetExtent(exts)

# make up some data
xaxis = np.linspace(-.5, 1., dims[0])
yaxis = np.linspace(-1.,1., dims[1])
zaxis = np.linspace(-1., .5, dims[2])
[xc,yc,zc] = np.meshgrid(zaxis,yaxis,xaxis, indexing="ij")
#data is time-dependent becasue we use a function of ts
data = .1*ts +sin(ts) + np.sqrt(xc**2 + yc**2 + zc**2, dtype='f')

output.PointData.append(data.ravel(), "scalarA")
"""

In [ ]:
programmableSource1 = ProgrammableSource()
programmableSource1.OutputDataSetType = 'vtkImageData'
programmableSource1.Script = RequestData
programmableSource1.ScriptRequestInformation = RequestInfo
programmableSource1.UpdatePipelineInformation()

In [ ]:
rep = Show()
rep.Representation = "Surface"
rep.ColorArrayName = ['POINTS', 'scalarA']

In [ ]:
view = GetRenderView()
view.CenterOfRotation = [11.5, 11.5, 11.5]
view.CameraPosition = [-23.571178826671265, 48.60210090801749, 74.13620782160231]
view.CameraFocalPoint = [11.5, 11.5, 11.5]
view.CameraViewUp = [0.12769986770432634, 0.8830512871745115, -0.45156745676336757]
view.CameraParallelScale = 19.91858428704209

from ipyparaview.widgets import PVDisplay
disp = PVDisplay(view)
w = display(disp)

In [ ]:
#SaveData('ts.vti', Writetimestepsasfileseries=1, Filenamesuffix='_%02d')